In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
torch.cuda.is_available()

True

In [4]:
t = torch.tensor([
    [[0, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0]],
    [[1, 0, 1, 0], [0, 1, 1, 1], [1, 1, 1, 1]],
], dtype=torch.long)
t.size()

torch.Size([2, 3, 4])

In [46]:
latent_dim = 32
emb = nn.Embedding(12, latent_dim)
emb._parameters["weight"].size()

torch.Size([12, 32])

In [47]:
nz = (t.reshape(2 * 3, 4) == 1).nonzero()
print(nz.dtype)
nz

torch.int64


tensor([[0, 3],
        [1, 1],
        [2, 0],
        [3, 0],
        [3, 2],
        [4, 1],
        [4, 2],
        [4, 3],
        [5, 0],
        [5, 1],
        [5, 2],
        [5, 3]])

In [52]:
batch_size = 2
n_steps = 3
chord_dim = 4
nz = (t.reshape(batch_size * n_steps, chord_dim) == 1).nonzero()
embedding = emb(nz[:, 1])
expand_index = nz[:, 0].repeat(latent_dim, 1).transpose(0, 1)
chord_filter = torch.zeros(batch_size * n_steps, latent_dim, dtype=torch.float, device="cpu").scatter_reduce(0, expand_index, embedding, reduce="sum")
chord_filter = F.tanh(chord_filter.reshape(batch_size, n_steps, latent_dim))
print(chord_filter.size())
chord_filter

torch.Size([2, 3, 32])


tensor([[[ 0.7494,  0.6090, -0.0744, -0.8256, -0.7599, -0.9524,  0.7652,
           0.8299,  0.4665, -0.7549,  0.9691,  0.6940, -0.5492,  0.6778,
           0.0093,  0.9131, -0.4749, -0.8238, -0.6540, -0.8483, -0.5127,
          -0.0151,  0.8002,  0.1865,  0.2520,  0.8869,  0.5657,  0.5510,
          -0.7548,  0.3551,  0.7456,  0.8163],
         [ 0.0927, -0.8706,  0.8783, -0.9351,  0.7961,  0.5267,  0.7198,
          -0.0888, -0.1457, -0.7328,  0.5399,  0.9045, -0.7991, -0.9149,
           0.5914, -0.5809, -0.6842,  0.5301, -0.8073,  0.8383,  0.4856,
          -0.2924,  0.8913, -0.5373,  0.4451, -0.1880, -0.2651,  0.0866,
           0.1447,  0.8980, -0.9038,  0.6857],
         [-0.0608, -0.1533,  0.8917,  0.7551,  0.3108, -0.2492, -0.0867,
           0.5447,  0.6108,  0.8948,  0.6592, -0.3702,  0.4602,  0.5060,
           0.8110,  0.3009, -0.4276, -0.4318, -0.0049, -0.5512,  0.1169,
           0.5857, -0.8945,  0.3297, -0.9927, -0.0535, -0.4953, -0.5299,
          -0.6824, -0.9546,  0

In [20]:
emb(torch.tensor([0, 1, 2], dtype=torch.long)).size()

torch.Size([3, 32])

In [44]:
embedding = torch.tensor([[1., 2., 3.], [4., 5., 6.], [7, 8, 9], [1., 2., 3.], [4., 5., 6.], [7, 8, 9]])
print(embedding.size())
index = torch.tensor([0, 1, 2, 3, 4, 0])
print(index.size())
expand_index = index.repeat(3, 1).transpose(0, 1)
print(expand_index.size())
input = torch.zeros(2 * 3, 3, dtype=torch.float)
print(input.size())
result = input.scatter_reduce(0, expand_index, embedding, reduce="sum")
print(result.size())
result.reshape(2, 3, 3)

torch.Size([6, 3])
torch.Size([6])
torch.Size([6, 3])
torch.Size([6, 3])
torch.Size([6, 3])


tensor([[[ 8., 10., 12.],
         [ 4.,  5.,  6.],
         [ 7.,  8.,  9.]],

        [[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [ 0.,  0.,  0.]]])